# Context Managers

We can say that in python context managers, manage the state surrounding a section of code, i.e open/close a connection.   
This is so we can verify that there is a code that will always run before and after our code inspite of a raising exception or an error.   

Context managers are basically doing two things:
1. enter - Enter the context. (`__enter__`)
2. exit - Exit the context. (`__exit__`)

```
with open("file.txt") as file:
    ...
```
- `with` is the keyword
- Here the `open` implements the context manager protocol.
- `as` stores what is returned from the `open` `__enter__` method, not the object `open()`

The context manager `exit` method works much like the `finally` on a `try/except`, it will always run even if an exception is raised.

Instead of a manager `Try/Except/Finally` is a good way to manage our resources since, it will run no matter what happens in the code block.
even if the exception is not handled if there is a finally it will run.

However it is cumbersome to write and a code full of catching errors is not clean, so enter context managers.

In [82]:
# we should keep in mind the finally clause of the try/except
import random

print("---- FINALLY ----")
try:
    i = random.randint(0, 1)
    10 / i
except ZeroDivisionError:
    print("i will sometimes run")
finally:
    print("i will always run")


print("---- FINALLY IN FUNCTION ----")
# The finally should always run even if a return statement is located in the exception
def my_func():
    try:
        i = random.randint(0, 1)
        10 / i
    except ZeroDivisionError:
        print("i will sometimes run")
        return
    finally:
        print("i will always run even if a return is on the except")

my_func()

---- FINALLY ----
i will always run
---- FINALLY IN FUNCTION ----
i will always run even if a return is on the except


We can see how the context manager, manages the connection to the file and how it closes after it finishes.

In [83]:
print("---- CONTEXT MANAGER ----")

with open('test.txt', 'w') as file:
    print(f"Inside context: file is closed -> {file.closed}")

print(f"Outside context: file is closed -> {file.closed}")

print("---- CONTEXT MANAGER FUNCTION----")

def test():
    with open("test.txt", "w") as file:
        print(f"Inside the context: file is closed -> {file.closed}")
        return file

file = test()
print(f"Outside the context: file is closed -> {file.closed}")

---- CONTEXT MANAGER ----
Inside context: file is closed -> False
Outside context: file is closed -> True
---- CONTEXT MANAGER FUNCTION----
Inside the context: file is closed -> False
Outside the context: file is closed -> True


## WITH

The keyword to instantiate/call the method for the context manager protocol.

- `with` has no local scope, it gets the scope of the one where it gets created.

In [56]:
print("---- SCOPE OF WITH ----")

with open('test.txt', 'w') as f:
    f.writelines('Hello world!')

with open('test.txt') as f:
    row = next(f)

print(f"we can access the value row: {row} from the last local manager.")

---- SCOPE OF WITH ----
we can access the value row: Hello world! from the last local manager.


## CONTEXT MANAGER PROTOCOL

The `__enter__` and `__exit__` methods are required to be implemented for a context manager to work.
- `__enter__(self)` runs when the context manager is called and it just returns any value, this is the one that gets assigned
on the `as` part of the `with` statement.
- `__exit__(self, exc_type, exc_value, exc_tb)` runs when the context manager finishes, it will always run, it must return either:
    - `True` if we handle the exception and don't want to raise it.
    - `False` if we want to raise the exception.

In [61]:
class MyContextManager:
    def __init__(self):
        self.obj = None

    def __enter__(self):
        print("Entering context ...")
        self.obj = 'The Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exec_tb):
        print('exiting context ....')
        if exc_type:
            print(f'*** Error Occurred: {exc_type}, {exc_value}')
        return True # False will raise the exception

ctx = MyContextManager()
with ctx as obj:
    print("Inside with block...")
    print(obj)
    raise ValueError('custom message....')

Entering context ...
Inside with block...
The Return Object
exiting context ....
*** Error Occurred: <class 'ValueError'>, custom message....


In [65]:
class Resource:
    def __init__(self, name):
        self.name = name
        self.state = None

class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None

    def __enter__(self):
        print('Entering context')
        self.resource = Resource(self.name)
        self.resource.state = 'created'

        return self.resource

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('Exiting Contest')
        self.resource.state = 'destroyed'
        if exc_type:
            print('error occured')
        return False


print("---- INSIDE WITH ----")
with ResourceManager('holi') as res:
    print(f"{res.name} = {res.state}")

print("---- OUTSIDE WITH ----")
print(f"{res.name} = {res.state}")
print(f'object res is still in our globals -> {"res" in globals()}')

---- INSIDE WITH ----
Entering context
holi = created
Exiting Contest
---- OUTSIDE WITH ----
holi = destroyed
object res is still in our globals -> True


In [71]:
print("---- ENTER RETURNS SELF ----")
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode

    def __enter__(self):
        print("opening file... ")
        self.file = open(self.name, self.mode)
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("close file")
        self.file.close()
        return False

with File("test.txt", "r") as f_ctx:
    print(next(f_ctx.file))


---- ENTER RETURNS SELF ----
opening file... 
Hello world!
close file


## Multiple Protocols

We can of course implement multiple protocols to our classes and not just the context manager one.

In [87]:
from itertools import islice

class DataIterator:
    def __init__(self, fname):
        self._fname = fname
        self._f = None

    def __enter__(self):
        self._f = open(self._fname)
        return self
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        if not self._f.closed is True:
            self._f.close()
        return False

    def __iter__(self):
        return self

    def __next__(self):
        row = next(self._f)
        return row.strip('\n').split(',')


di_ctx = DataIterator("nyc_parking_tickets_extract.csv")

with di_ctx as file:
    print(f"First 3 lines of our class -> {list(islice(file, 2))}")

First 3 lines of our class -> [['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description'], ['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']]


## Uses

OPEN / CLOSE
- a file
- a socket

START / STOP
- db transaction
- timer

In [116]:
print("---- CONTEXT MANAGER IMPLEMENTATION ----")

from time import perf_counter, sleep

class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start

with Timer() as timer:
    sleep(1)

print(f"The with block took {timer.elapsed} seconds to run")

print("---- CONTEXT MANAGER DECORATOR ----")
from contextlib import contextmanager

@contextmanager
def timer():
    stats = dict()
    start = perf_counter()
    stats['start'] = start
    try:
        print("yielding .....")
        yield stats
    finally:
        end = perf_counter()
        stats['end'] = end
        stats['elapsed'] = end - start

with timer() as t:
    sleep(2)

print(f"The with block took {t['elapsed']} seconds to run")

---- CONTEXT MANAGER IMPLEMENTATION ----
The with block took 1.0016097170009743 seconds to run
---- CONTEXT MANAGER DECORATOR ----
yielding .....
The with block took 2.0025690619950183 seconds to run


LOCK / RELEASE
- thread lock

CHANGE / RESET
- decimal context precision
- stdout to a file

In [93]:
import decimal

print("---- DECIMAL CONTEXT ----")
print(f"the context for the default decimal is -> {decimal.getcontext()}")

print(f"We can modify it globaly, prec before -> {decimal.getcontext().prec}")
decimal.getcontext().prec = 4
print(f"We can modify it globaly, prec after -> {decimal.getcontext().prec}")

print("---- DECIMAL CONTEXT CLASS ----")
# so instead of creating a variable to store the old value we can manage a context with the context manager
class precision:
    def __init__(self, prec):
        self._prec = prec
        self._current_prec = decimal.getcontext().prec

    def __enter__(self):
        decimal.getcontext().prec = self._prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self._current_prec
        return False

print(f"decimal precision context manager, prec before context -> {decimal.getcontext().prec}")
with precision(10):
    print(f"decimal precision context manager, prec inside context -> {decimal.getcontext().prec}")
print(f"decimal precision context manager, prec after context -> {decimal.getcontext().prec}")

print("---- DECIMAL CONTEXT BUILT IN ----")
print(f"decimal precision context manager, prec before context -> {decimal.getcontext().prec}")
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(f"decimal precision context manager, prec inside context -> {decimal.getcontext().prec}")
print(f"decimal precision context manager, prec after context -> {decimal.getcontext().prec}")

---- DECIMAL CONTEXT ----
the context for the default decimal is -> Context(prec=4, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
We can modify it globaly, prec before -> 4
We can modify it globaly, prec after -> 4
---- DECIMAL CONTEXT CLASS ----
decimal precision context manager, prec before context -> 4
decimal precision context manager, prec inside context -> 10
decimal precision context manager, prec after context -> 4
---- DECIMAL CONTEXT BUILT IN ----
decimal precision context manager, prec before context -> 4
decimal precision context manager, prec inside context -> 3
decimal precision context manager, prec after context -> 4


In [120]:
print("---- CONTEXT MANAGER IMPLEMENTATION ----")
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout

    def __enter__(self):
        self._file = open(self._fname, "w")
        sys.stdout = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False

with OutToFile("stdout.txt"):
    print("oh my god!")

print("---- CONTEXT MANAGER DECORATOR ----")
from contextlib import contextmanager

@contextmanager
def out_to_file(fname):
    current_stdout = sys.stdout
    file = open(fname, "w")
    sys.stdout = file
    try:
        yield
    finally:
        file.close()
        sys.stdout = current_stdout

with out_to_file("stdout_dec.txt"):
    print("oh my god!")

print("---- CONTEXT MANAGER BUILTIN ----")
from contextlib import redirect_stdout

with open("stdout_builtin.txt", "w") as f:
    with redirect_stdout(f):
        print("oh mi Dios!!")

---- CONTEXT MANAGER IMPLEMENTATION ----
---- CONTEXT MANAGER DECORATOR ----
---- CONTEXT MANAGER BUILTIN ----


WACKY STUFF
- html tags
- list maker

In [101]:
class Tag:
    def __init__(self, tag):
        self._tag = tag

    def __enter__(self):
        print(f"<{self._tag}>", end="")

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f"</{self._tag}>", end="")
        return False

with Tag('p'):
    print("hello ", end="")
    with Tag('b'):
        print("hello from inside", end="")
    print(" world!!", end="")

<p>hello <b>hello from inside</b> world!!</p>

In [104]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)

    def __enter__(self):
        self._current_indent += self._indent
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent

    def print(self, arg):
        print(f"{' ' * self._current_indent} {self._prefix} {arg}")

with ListMaker("my awesome list", "> ", indent=4) as list_ctx:
    list_ctx.print("today")
    with list_ctx:
        list_ctx.print("write code")
    list_ctx.print("tomorrow")
    with list_ctx:
        list_ctx.print("debug code")
    

my awesome list
     >  today
         >  write code
     >  tomorrow
         >  debug code


## Generators and Context Managers

When creating the context managers, one thing Python wanted to do was to generate this generator functions like context managers.   

This way we can pass a generator fucntion that generates a context and we create the generic protocol inside a class.

In [108]:
# Pattern for generator function
def my_gen(f_name, mode):
    f = open(f_name, mode)
    try:
        print('creating context and yielding object')
        yield f
    finally:
        print('exiting context and cleaning up') 

class GenContextManager:
    def __init__(self, gen_func, *args, **kwargs):
        self._gen_func = gen_func(*args, **kwargs)

    def __enter__(self):
        return next(self._gen_func)

    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen_func)
        except StopIteration:
            pass
        return False


with GenContextManager(my_gen, 'nyc_parking_tickets_extract.csv', mode="r") as obj:
    print(obj)
        

creating context and yielding object
<_io.TextIOWrapper name='nyc_parking_tickets_extract.csv' mode='r' encoding='UTF-8'>
exiting context and cleaning up


There is however a lot of cleaning up to do here, since we may receive or get a lot of error handling when implementing a solution.   
That is why there is a built in solution for these cases.   
To work with the built - in case we need the generator function to be of a certain design.

```
def gen(args):
    # set up happens here, or inside try
    try:
        yield obj # Whatever normally gets returned by __enter__
    finally:
        # perform clean up code here
```

In [114]:
print("---- DECORATOR IMPLEMENTATION ----")

class GenContextManager:
    def __init__(self, gen):
        self.gen = gen

    def __enter__(self):
        print("calling next to get the yielded value from the generator")
        return next(self.gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("calling next to perform cleanup in generator")
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False

def context_manager_dec(gen_fn):
    def helper(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        ctx = GenContextManager(gen)
        return ctx
    return helper

@context_manager_dec
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print("closing file...")
        f.close()

with open_file('test.txt') as f:
    print(f.readlines())
 

print("---- DECORATOR BUILT IN ----")

from contextlib import contextmanager

@contextmanager
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print("closing file...")
        f.close()

with open_file('test.txt') as f:
    print(f.readlines())


---- DECORATOR IMPLEMENTATION ----
calling next to get the yielded value from the generator
opening file...
['holi\n', 'holi\n', 'holiii\n', 'holiiiiiiiii']
calling next to perform cleanup in generator
closing file...
---- DECORATOR BUILT IN ----
opening file...
['holi\n', 'holi\n', 'holiii\n', 'holiiiiiiiii']
closing file...


## Nested Context Managers

In [132]:
from contextlib import contextmanager
from tkinter import E

print("---- CONTEXT MANAGER NESTED ----")

@contextmanager
def open_file(f_name):
    print(f"opening file {f_name}")
    f = open(f_name)
    try:
        yield f
    finally:
        print(f"closing file {f_name}")
        f.close()

f_names = 'file1.txt', 'file2.txt', 'file3.txt'

class NestedContexts:
    def __init__(self, *contexts):
        self._enters = []
        self._exits = []
        self._values = []

        for ctx in contexts:
            self._enters.append(ctx.__enter__)
            self._exits.append(ctx.__exit__)

    def __enter__(self):
        for enter in self._enters:
            self._values.append(enter())
        return self._values
        
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_tb)

with NestedContexts(open_file(f_names[0]), open_file(f_names[1]), open_file(f_names[2])) as f:
    for file in f:
        print(list(file))

print("---- CONTEXT MANAGER STACK ----")

class NestedContexts:
    def __init__(self):
        self._exits = []

    def __enter__(self):
        return self
       
    def enter_context(self, ctx):
        self._exits.append(ctx.__exit__)
        value = ctx.__enter__()
        return value

    def __exit__(self, exc_type, exc_value, exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_tb)
        return False

with NestedContexts() as stack:
    files = [stack.enter_context(open_file(f)) for f in f_names]

print("---- CONTEXT MANAGER EXIT STACK BUILTIN ----")
from contextlib import ExitStack

with ExitStack() as stack:
    files = [stack.enter_context(open_file(f)) for f in f_names]

closing file file1.txt
closing file file3.txt
closing file file2.txt
---- CONTEXT MANAGER NESTED ----
opening file file1.txt
opening file file2.txt
opening file file3.txt
['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']
['file3_line1\n', 'file3_line2\n', 'file3_line3']
closing file file3.txt
closing file file2.txt
closing file file1.txt
---- CONTEXT MANAGER STACK ----
opening file file1.txt
opening file file2.txt
opening file file3.txt
closing file file3.txt
closing file file2.txt
closing file file1.txt
---- CONTEXT MANAGER EXIT STACK BUILTIN ----
opening file file1.txt
opening file file2.txt
opening file file3.txt
closing file file3.txt
closing file file2.txt
closing file file1.txt


## Caveats with Lazy Iterators

When returning a lazy iterator from within a context manager the resource will be closed by the time it is started iteration.

In [81]:
import csv

print("---- CAVEAT ----")

def read_data():
    with open("nyc_parking_tickets_extract.csv") as f:
        return csv.reader(f, delimiter=',', quotechar='"') # we return a lazy iterator
reader = read_data()

try:
    next(reader)
except ValueError as e:
    print("not working, the operation is on a closed file ->", e)

print("---- SOLUTION: YIELD FROM ----")

def read_data_from():
    with open("nyc_parking_tickets_extract.csv") as f:
        yield from csv.reader(f, delimiter=',', quotechar='"') # yield from will yield from within the with block and will still be lazy.

reader = read_data_from()
print(f"we can iterate now -> {next(reader)}")

---- CAVEAT ----
not working, the operation is on a closed file -> I/O operation on closed file.
---- SOLUTION: YIELD FROM ----
we can iterate now -> ['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']


# Exercise

## Goal 1

Create a context manager that only requires file name and provides us an iterator we can use to iterate over the
data in those files.

- Create a class that implements the context manager protocol (single class)
- yields a named tuple, all fields can be thought of as strings

In [181]:
import csv
from collections import namedtuple
from itertools import islice

def get_dialect(f_name):
    with open(f_name) as f:
        return csv.Sniffer().sniff(f.read(1000))

class FileReader:
    def __init__(self, fname):
        self.fname = fname
        self.file = None
   
    def __enter__(self):
        self.file = open(self.fname)
        self.rows = csv.reader(self.file, dialect=get_dialect(self.fname))
        headers = map(lambda x: x.lower(), next(self.rows))
        self._row = namedtuple("Row",  headers) 
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        if not self.file.closed:
            self.file.close()
        return False

    def __next__(self):
        if self.file.closed:
            raise StopIteration
        return self._row(*next(self.rows))
                    
    def __iter__(self):
        return self
        


with FileReader('cars.csv') as data:
    for row in islice(data, 10):
        print(row)

with FileReader('personal_info.csv') as data:
    for row in islice(data, 10):
        print(row)

Row(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
Row(car='Buick Skylark 320', mpg='15.0', cylinders='8', displacement='350.0', horsepower='165.0', weight='3693.', acceleration='11.5', model='70', origin='US')
Row(car='Plymouth Satellite', mpg='18.0', cylinders='8', displacement='318.0', horsepower='150.0', weight='3436.', acceleration='11.0', model='70', origin='US')
Row(car='AMC Rebel SST', mpg='16.0', cylinders='8', displacement='304.0', horsepower='150.0', weight='3433.', acceleration='12.0', model='70', origin='US')
Row(car='Ford Torino', mpg='17.0', cylinders='8', displacement='302.0', horsepower='140.0', weight='3449.', acceleration='10.5', model='70', origin='US')
Row(car='Ford Galaxie 500', mpg='15.0', cylinders='8', displacement='429.0', horsepower='198.0', weight='4341.', acceleration='10.0', model='70', origin='US')
Row(car='Chevrolet Impala', mpg='14.0', cy

## Goal 2

Re-implement goal 1 using a generator function instead

- use the `@contextmanager` from `contextlib`

In [186]:
from contextlib import contextmanager



@contextmanager
def file_reader(f_name):
    
    f = open(f_name)
    try:
        sniff = csv.Sniffer().sniff(f.read(1000))
        f.seek(0)
        reader = csv.reader(f, sniff)
        headers = map(lambda x: x.lower(), next(reader))
        Row = namedtuple("Row",  headers)
    
        yield (Row(*row) for row in reader)
    finally:
        f.close()

with file_reader('cars.csv') as data:
    for row in islice(data, 10):
        print(row)

with file_reader('personal_info.csv') as data:
    for row in islice(data, 10):
        print(row)

Row(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
Row(car='Buick Skylark 320', mpg='15.0', cylinders='8', displacement='350.0', horsepower='165.0', weight='3693.', acceleration='11.5', model='70', origin='US')
Row(car='Plymouth Satellite', mpg='18.0', cylinders='8', displacement='318.0', horsepower='150.0', weight='3436.', acceleration='11.0', model='70', origin='US')
Row(car='AMC Rebel SST', mpg='16.0', cylinders='8', displacement='304.0', horsepower='150.0', weight='3433.', acceleration='12.0', model='70', origin='US')
Row(car='Ford Torino', mpg='17.0', cylinders='8', displacement='302.0', horsepower='140.0', weight='3449.', acceleration='10.5', model='70', origin='US')
Row(car='Ford Galaxie 500', mpg='15.0', cylinders='8', displacement='429.0', horsepower='198.0', weight='4341.', acceleration='10.0', model='70', origin='US')
Row(car='Chevrolet Impala', mpg='14.0', cy